In [1]:
import re

from konlpy.tag import Hannanum, Okt, Kkma, Komoran
hannanum = Hannanum()
okt = Okt()
kkma = Kkma()
komoran = Komoran()

## 추가할 것들
1. 한글에 대해서만 토크나이저 적용하기
2. \ 를 제거 - 어차피 몇 개 없으니까 무시?


## 소개
1. 아래에 구두점 제거는 띄어쓰기로 대체함을 의미, 띄어쓰기가 필요없는 구간은 ''으로 교체 (삭제)  
2. 수학식 공백 제거 이전에 숫자 변환해주는 게 굿  

## 보완할 점  
1. 각도에 대해서는 숫자 보존?  
2. math_num6 에서 차수가 3보다 크면 #으로 할지 공백으로 할지 어떡하나~  
3. math_num8 숫자 제거 후 그 자리를 공백으로 채울지 아니면 공백마저 없는 게 좋을지

## 한글 표현 관련 kor_

#### Punctuation punc
1. 모든 구두점 보존  
2. 한글 부분에서 온점, 반점만 제거 (느낌표 추가)
3. 한글 부분에서 모든 구두점 제거 : 물음표까지 싹다 
  - 하지만 수학 공식들을 두르는 작은 따옴표 보존
  - 괄호 보존
4. 한글 부분에서 모든 구두점 제거, 괄호까지 제거
5. 긴혹 나타나는 특수기호까지 제거 <-- 이건 없는걸로 딱히 필요 없을듯  

#### Language lang
1. 모든 한글 제거  

## 수학 표현 관련 math_
#### 숫자 관련  num
1. 숫자 보존  
2. 차수에 대한 형식 통일 : ^(-1) --> ^-1 : 이후 차수 따로 다루려면 형식 통일해야 함
 - (ax+1) 또는 (alpha-1) 은 그대로 보존
 - 형식 통일을 ^-1 --> ^(-1)으로 할지 고려중  
     - 하지만 캐릭터가 많아지는 건 맘에 안듦
3. 차수만 보존, 나머지 숫자는 '#'으로 변환 
4. 차수만 보존, 나머지 숫자 제거
5. 차수 중에서 <=3 까지만 보존, 나머지 다 '#'으로 변환 (5.1 로 버그 고침)
6. 차수 중에서 <=3 까지만 보존, 나머지 숫자 제거 (6.1 로 버그 고침)
7. 모든 숫자 '#'으로 변환  
8. 모든 숫자 제거  

#### 기호 관련 sym
1.  div divide '/' 를 '/'로 통일  
2. xx * 를 * 로 통일
 - xxx는 * x 로 변환  
 - times 도 * 로 변환
3. 수학 공식 내에 공백 제거
 - 3.1 모든 수학적 표현으로 추측되는 부분에 공백 제거
4. 수학 공식 그냥 싹다 제거
 - 4.1 for noisy text
5. 수학 공식을 두르는 ` 제거  
6. 모든 괄호 유형 통일 하지만 {::}에 대해서는 유지  

## 토크나이저 (띄어쓰기로 우선 구분 후 토큰 반환) token_

#### 한글 토크나이져 관련 kor
1. Komoran 토크나이져로 한글 부분에만 적용 후 띄어쓰기 상태로 반환  

#### 띄어쓰기 관련 split
1. 연산자에 대해서 띄어쓰기(연산자에 '@' 도 추가)  
2. dict_sample1 에 대해서 다 띄어쓰기 해주기  
 - dict_sample3 에 대해서도...
3. 숫자 # 에 대해서 띄어쓰기  
4. 차수에 대해서 정리  (f^-1 , g^-1 있을 수 있도록 얘네들은 특별히 봐줄까?)
5. {::}_ 도 하나의 토큰으로 {::}와 구분  
6. 그냥 숫자에 대해서도 띄어쓰기 보장  

## 노이즈 잡기 noise_  

#### 없애기 del  
1. 기타 기호 없앰  
2. \ 기호 없앰  
3. tarc 제거  
4. 잡 char 제거  
5. begin{cases} end{cases} 제거 할까 말까 ㅎㅎ  
6.  {::} 제거  -  이거는 open data 에 해당  
7. _ 제거 - 둘 다 해당

#### 변환해주기 substitution sub  
1. dfrac 잡기 '/' 를 적절한 위치에 삽입 해주깅
2. dict_sample2 토대로 변환
3. dict_sample3 역변환  
4. 간혹 이상한 숫자 형식에 대해서 통일  

## 추가적인 것들 extra
1. 괄호 안에 무의미한 공백 제거
2. 연속적인 공백 제거
3. </br>, <legend>, <field/> 등등 제거 ㅋㅋㅋ 이건 클릭해서 보세요  

## 추가적인 것들 extra  
4. 괄호 안에 공백 보장

In [ ]:
# noise delete 6 : {::} 제거  -  이거는 open data 에 해당
def noise_del6(edit_text):
    
    edit_text = re.sub('\{\:\:\}', ' ', edit_text)
    
    return edit_text


# noise delete 7 : _ 제거 - 둘 다 해당

In [3]:
# math_sym3 에 쓰임
def remove_space(matchObj):
    words = matchObj.group().split(" ")
    #print(words)
    return "".join(words)


# math_num5~6 에 쓰임
def power(matchObj):
    matchObj = re.search('-?\d+', matchObj.group()).group()
    if abs(int(matchObj)) > 3:
        matchObj = '#'
    return '^' + matchObj

# tok_kor1 에 쓰임
def tok_komoran(matchObj):
    matchObj = komoran.morphs(matchObj.group())
    return " ".join(matchObj)


In [36]:
# for open data
dict_sample1 = ['sin', 'cos', 'tan', 'theta',
                'sqrt',
                'alpha', 'beta', 'gamma', 'delta', 'omega', 'phi', 'sigma',
                'bar',
                'cdots', 'vdots',
               'rarr',
               'pi',
               'emptyset',
               'therefore,',
               'triangle', 'square', 'angle', 'Delta',
               'to',
               'uu', 'nn', 'sub',
                'root', 
                'lim_', '->', 'oo',
               'sum_', 'log', 
               '▼', '△', '/_']

# 'in' 은 이미 띄어쓰기가 되어있을거야

# for test hidden data 2
dict_sample2_1 = [('α', 'alpha'), ('β', 'beta'),  ('γ', 'gamma'), ('σ', 'sigma'), ('θ','theta')] 
dict_sample2_2 = [('left(', '('), ('right)', ')'), ('geq', '>='), ('leq', '<=')]
dict_sample2_3 = [('^{°}','°')] # "°"
dict_sample2_4 = [('∠','/_')] # /_ 역변환도 ㄱㅊ하지 않을까


In [150]:
# for test hidden data 2

# 최대한 짧은 쪽으로 변환

# patter 형식으로 약간씩 수정
dict_sample2_1 = [('infty', 'oo')] # 얘는 그닥 안나와서 중요하려나? hidden 2 에서는 몇 번 나오는데 OCR일 가능성 높음
dict_sample2_2 = [('left\(', '('), ('right\)', ')'), ('geq', '>='), ('leq', '<=')]
dict_sample2_3 = [('\^\{°\}','°')] # "°"


# 역변환 용
dict_sample2_4 = [('α', 'alpha'), ('β', 'beta'),  ('γ', 'gamma'), ('σ', 'sigma'), ('θ','theta')] 
dict_sample2_5 = [('∠','\/\_')] # /_ 역변환도 ㄱㅊ하지 않을까
dict_sample2_6 = [('→', '\-\>')] # 역변환 ㄱ?
dict_sample2_7 = [('∩', 'nn'), ('∪', 'uu')] # 역변환 ㄱㄱ
dict_sample2_8 = [('∩', 'cap')] # 이것도 역변환
dict_sample2_9 = [('Ø', 'emptyset')]
# hidden2에 AnB 하나 있지만 이건 딱 하나니 버리자~~!@#

# 하지만 bert같은 곳에서는 vocab 이 있는지 없는지에 대해서도 중요할듯

In [155]:
dict_sample3 = dict_sample2_4 + dict_sample2_5 + dict_sample2_6 + dict_sample2_7 + dict_sample2_8 + dict_sample2_9

In [156]:
dict_sample2 = dict_sample2_1 + dict_sample2_2 + dict_sample2_3

In [153]:
dict_number = [('①', '(1)'), ('②', '(2)'), ('③', '(3)'), ('④', '(4)'), ('⑤', '(5)')]

In [134]:
dict_delete = ['★', '○']

In [135]:
dict_case = ['begin\{cases\}', 'end\{cases\}']

In [5]:
# extra1 : 괄호 안에 공백 제거
# () [] {} || 형식의 괄호에 대해서 적용
def extra1(edit_text):
    
    edit_text = re.sub('\(\s+','(', edit_text)   
    edit_text = re.sub('\s+\)',')', edit_text)
    
    edit_text = re.sub('\[\s+','[', edit_text)   
    edit_text = re.sub('\s+\]',']', edit_text)
    
    edit_text = re.sub('\{\s+','{', edit_text)   
    edit_text = re.sub('\s+\}','}', edit_text)
    
    edit_text = re.sub('\|\s+','|', edit_text)   
    edit_text = re.sub('\s+\|','|', edit_text)

    return edit_text

# extra2 : 연속적인 공백 제거 
def extra2(edit_text):
    
    edit_text = re.sub('\s+', ' ', edit_text)
    
    return edit_text

# extra3 : <br/> 같은 거 다 제거
def extra3(text):

    edit_text = re.sub('<[-\s"/=A-Za-z]+?>', '', text)  
    
    return edit_text


# extra4 : 괄호 안밖으로 공백 보장
# () [] {} || 형식의 괄호에 대해서 적용
def extra4(edit_text):
    
    edit_text = re.sub('\((?P<word>\S)',' ( \g<word>', edit_text)   
    edit_text = re.sub('(?P<word>\S)\)','\g<word> ) ', edit_text)

    edit_text = re.sub('\[(?P<word>\S)',' [ \g<word>', edit_text)   
    edit_text = re.sub('(?P<word>\S)\]','\g<word> ] ', edit_text)

    edit_text = re.sub('\{(?P<word>\S)',' { \g<word>', edit_text)   
    edit_text = re.sub('(?P<word>\S)\}','\g<word> } ', edit_text)

    edit_text = re.sub('\|(?P<word>\S)',' | \g<word>', edit_text)   
    edit_text = re.sub('(?P<word>\S)\|','\g<word> | ', edit_text)
    
    # 연속적인 공백 제거
    edit_text = extra2(edit_text)
    
    # {::} 는 보존할 수 있도록~
    edit_text = re.sub('\{ :', '{:', edit_text)
    edit_text = re.sub(': \}', ':}', edit_text)

    return edit_text

In [279]:
# korean punctuation1 : 모든 구두점 보존

def kor_punc1(edit_text):
    
    # 그냥 숫자 채우고 싶어서 만들었습니다 :)

    return edit_text

# korean punctuation2 : 한글 부분에서 온점 반점만 제거 (느낌표 축가)
# 한글 단어 끝날 때와
# 수학적 표현이 끝났을 때에 있는 구두점들에 대해서만
def kor_punc2(edit_text):
    
    # (한글)단어 끝날 때 있는 . , 제거
    edit_text = re.sub('(?P<word>[가-힣ㄱ-ㅎㅏ-ㅣ]+)(\s+)?([.,!]+)?', '\g<word>', edit_text)

    # 수학식 내부 말고 외부에 있는 . , 제거
    edit_text = re.sub('`(\s+)?([.,!]+)?', '`', edit_text)
    
    # 연속으로 있는 공백 하나로 수정
    #edit_text = re.sub('\s+', ' ', edit_text)

    return edit_text

# korean punctuation3 : 한글 부분에서 모든 구두점 제거

def kor_punc3(edit_text):
    
    # (한글)단어 뒤에 있는 구두점 제거
    edit_text = re.sub('(?P<word>[가-힣ㄱ-ㅎㅏ-ㅣ]+)(\s+)?([.,?!<>-]+)?', '\g<word> ', edit_text)
    
    # (한글)단어 앞에 있는 구두점 제거
    edit_text = re.sub('([.,?!<>-]+)?(\s+)?(?P<word>[가-힣ㄱ-ㅎㅏ-ㅣ]+)', ' \g<word>', edit_text)
    
    # 수학식 내부 말고 외부에 있는 . , 제거
    edit_text = re.sub('`(\s+)?([.,?!<>-]+)?', '` ', edit_text)
    
    # 연속으로 있는 공백 하나로 수정
    edit_text = re.sub('\s+', ' ', edit_text)
    
    # 괄호 안에 공백이 띄어지게 돼서, 이를 재차 수정 :)
    edit_text = extra1(edit_text)

    return edit_text

# korean punctuation4 : 한글 부분에서 모든 구두점 제거 (괄호 포함)

def kor_punc4(edit_text):
    
    # (한글)단어 뒤에 있는 구두점 제거
    edit_text = re.sub('(?P<word>[가-힣ㄱ-ㅎㅏ-ㅣ]+)(\s+)?([\]\[.,?!|(){}<>-]+)?', '\g<word> ', edit_text)
    
    # (한글)단어 앞에 있는 구두점 제거
    edit_text = re.sub('([\]\[().,?!{}|<>-]+)?(\s+)?(?P<word>[가-힣ㄱ-ㅎㅏ-ㅣ]+)', ' \g<word>', edit_text)
    
    # 수학식 내부 말고 외부에 있는 . , 제거
    edit_text = re.sub('`(\s+)?([\]\[.,?!|{}<>-]+)?', '` ', edit_text)
    
    # 연속으로 있는 공백 하나로 수정
    edit_text = re.sub('\s+', ' ', edit_text)

    return edit_text


In [280]:
# 모든 한글 지워보기
def kor_lang1(edit_text):
    
    edit_text = re.sub('[가-힣ㄱ-ㅎㅏ-ㅣ]', '', edit_text)
    edit_text = re.sub('`', '', edit_text)
    
    return edit_text

In [220]:
# math number1 : 모든 숫자 그대로 보존

def math_num1(edit_text):
    
    # 그냥 숫자 채우고 싶어서 만들었습니다 :)
    
    return edit_text

# math number2 : 차수에 대한 형식 통일

def math_num2(edit_text):
    
    # 차수가 괄호 안에 있으면 괄호 제거
    edit_text = re.sub('\^\(?(?P<exp>-?\d+)\)?', '^\g<exp>', edit_text)
    
    return edit_text

# math number3 : 차수는 보존, 나머지 숫자 '#' 로 통일

def math_num3(edit_text):
    
    # 앞 두 숫자는 변환이 안돼서 따로 해준다
    edit_text_front = re.sub('\d+', '#', edit_text[0:2])
    
    # 두번 하는 이유:
    # 숫자가 읽혀서 차수 아닌 걸 판별하는 데 방해가 되어서
    # 첫번째로 하면 방해되는 숫자는 #로 변환되어서
    # 두번째 잘 변환되지만, -가 붙혀진 숫자에 대해서는 변환이 안되기에
    # 세번째까지 필요하다
    edit_text = re.sub('(?P<sym>[\s^^0-9][^^0-9-])\-?\d+', '\g<sym>#', edit_text_front + edit_text[2:])
    edit_text = re.sub('(?P<sym>[^^0-9][^^0-9-])\-?\d+', '\g<sym>#', edit_text)
    edit_text = re.sub('(?P<sym>[^^0-9][^^0-9-])\-?\d+', '\g<sym>#', edit_text)

    return edit_text

# math number4 : 차수는 보존, 나머지 숫자 제거

def math_num4(edit_text):
    
    # 앞 두 숫자는 변환이 안돼서 따로 해준다
    edit_text_front = re.sub('\d+', '#', edit_text[0:2])
    
    # 두번 하는 이유:
    # 숫자가 읽혀서 차수 아닌 걸 판별하는 데 방해가 되어서
    # 첫번째로 하면 방해되는 숫자는 #로 변환되어서
    # 두번째 잘 변환되지만, -가 붙혀진 숫자에 대해서는 변환이 안되기에
    # 세번째까지 필요하다
    edit_text = re.sub('(?P<sym>[\s^^0-9][^^0-9-])\-?\d+', '\g<sym>', edit_text)
    edit_text = re.sub('(?P<sym>[^^0-9][^^0-9-])\-?\d+', '\g<sym>', edit_text)
    edit_text = re.sub('(?P<sym>[^^0-9][^^0-9-])\-?\d+', '\g<sym>', edit_text)
    edit_text_front = re.sub('\d+', '', edit_text[0:2])

    return edit_text_front + edit_text[2:]

# math number5 : 차수는 보존 (<= 3), 나머지 숫자 '#' 로 통일

def math_num5(edit_text):

    edit_text = re.sub('\^\(?(?P<exp>-?\d+)\)?', '^\g<exp>', edit_text)
    edit_text = re.sub('\^-?\d+',  repl=power, string=edit_text)
    edit_text = re.sub('(?P<sym>[^\^])\d+', '\g<sym>#',edit_text)
    
    return edit_text

# math number6 : 차수는 보존 (<= 3), 나머지 숫자 제거
# 일단 3보다 큰 숫다 #로 대체

def math_num6(edit_text):

    edit_text = re.sub('\^\(?(?P<exp>-?\d+)\)?', '^\g<exp>', edit_text)
    edit_text = re.sub('\^-?\d+',  repl=power, string=edit_text)
    edit_text = re.sub('(?P<sym>[^\^])\d+', '\g<sym>',edit_text)
    
    return edit_text

# math number6 : 모든 숫자 '#' 로 대체

def math_num7(edit_text):

    edit_text = re.sub('\d+', '#', edit_text)
    
    return edit_text

# math number8 : 모든 숫자 제거
# 숫자가 있던 자리에는 아무것도 남지 않는다

def math_num8(edit_text):

    edit_text = re.sub('\d+', '', edit_text)
    
    return edit_text

# math number 5.1 : math number 5 에서 ^-1에 대해 잘못 변환되는 거 고침
def math_num5_1(edit_text):
    
    # 일단 차수 제외 모든 숫자 # 변환
    edit_text = math_num3(edit_text)
    
    edit_text = re.sub('\^-?\d+',  repl=power, string=edit_text)
    
    return edit_text

# math number 6.1 : math number 5 에서 ^-1에 대해 잘못 변환되는 거 고침
def math_num6_1(edit_text):
    
    # 일단 차수 제외 모든 숫자 제거
    edit_text = math_num4(edit_text)
    
    edit_text = re.sub('\^-?\d+',  repl=power, string=edit_text)
    
    return edit_text

In [433]:
# math symbol1 : 나눗셈에 대한 형식 통일 : '/' 으로

def math_sym1(edit_text):

    edit_text = re.sub('divide|div', '/', edit_text)
    
    return edit_text

# math symbol2 : 곱셈에 대한 형식 통일 : '*'' 으로

def math_sym2(edit_text):
    
    # *x 도 헷걸라지 않게 미리 변환
    edit_text = re.sub('xxx', '*x', edit_text)
    edit_text = re.sub('xx', '*', edit_text)
    edit_text = re.sub('times', '*', edit_text)
    
    return edit_text

# math symbol3 : 수학 공식 내에 빈칸 제거

def math_sym3(edit_text):
    
    # 수학식 공백 다 없애주기
    edit_text = re.sub('`[^가-힣]+?`', repl=remove_space, string=edit_text) 
    
    return edit_text

# math symbol4 : 수학 공식 그냥 제거
# 그 자리에 ` ` 는 남도록 했지만 math_sym5 로 쉽게 제거
# 영어도 사라짐....

def math_sym4(edit_text):
    
    edit_text = re.sub('`[^가-힣ㄱ-ㅎㅏ-ㅣ]+?`', '` `', edit_text) 
    
    return edit_text

# math symbol5 : 수학 공식을 두르는 ` ` 제거

def math_sym5(edit_text):
    
    edit_text = re.sub('`', ' ',edit_text)
    
    return edit_text

# math symbol6 : 모든 유형의 괄호 통일 ({::} 에 대해서는 조심~)

def math_sym6(edit_text):
    
    edit_text = re.sub('{(?P<sym>[^:])', '(\g<sym>', edit_text)
    edit_text = re.sub('(?P<sym>[^:])}', '\g<sym>)', edit_text)
    edit_text = re.sub('{(?P<sym>[^:])', '(\g<sym>', edit_text)
    edit_text = re.sub('(?P<sym>[^:])}', '\g<sym>)', edit_text)
    edit_text = re.sub('\[', '(', edit_text)
    edit_text = re.sub('\]', ')', edit_text)
    
    return edit_text

# math_symbol7 : 괄호 제거

def math_sym7(edit_text):
    
    edit_text = re.sub('\(|\)', ' ', edit_text)
    
    return edit_text

# math symbol3-1 : 수학 공식 내에 빈칸 제거 '`' 없을 때에도 적용 가능하도록

def math_sym3_1(edit_text):
    
    # 수학식 공백 다 없애주기
    edit_text = re.sub('([^가-힣ㄱ-ㅎㅏ-ㅣ][^가-힣ㄱ-ㅎㅏ-ㅣ])+?', repl=remove_space, string=edit_text) 
    
    return edit_text

# math symbol4 : 수학 공식 그냥 제거 (for noisy text)
# 그 자리에 $ $ 는 남도록 했지만 math_sym5 로 쉽게 제거
# 영어도 사라짐.... 아니면 수학 외부에 대한 것은 보존 가능할지도.12.3!@/3?
def math_sym4_1(edit_text):
    
    edit_text = re.sub('\$[^가-힣ㄱ-ㅎㅏ-ㅣ]+?\$', '$ $', edit_text) 
    
    return edit_text

In [615]:
# tokenizer korean 1 : 한국어에 대해서 komoran 토크나이져 적용
def tok_kor1(edit_text):
    
    # 띄어쓰기 양 옆에 삽입
    edit_text = re.sub('[가-힣][가-힣\s]+', repl = tok_komoran ,string = edit_text)
    
    # 한글이랑 다른 기호, 영어, 숫자랑 연속되면 띄어쓰기로 구분!
    edit_text = re.sub('(?P<kor>[가-힣ㄱ-ㅎㅏ-ㅣ])(?P<notkor>[^가-힣ㄱ-ㅎㅏ-ㅣ])', '\g<kor> \g<notkor>', edit_text)
    
    # 연속적인 공백 제거
    edit_text = extra2(edit_text)
    
    return edit_text

def tok_kor2(edit_text):
    
    # 한글이랑 다른 기호, 영어, 숫자랑 연속되면 띄어쓰기로 구분!
    edit_text = re.sub('(?P<kor>[가-힣ㄱ-ㅎㅏ-ㅣ])(?P<notkor>[^가-힣ㄱ-ㅎㅏ-ㅣ])', '\g<kor> \g<notkor>', edit_text)
    edit_text = re.sub('(?P<notkor>[^가-힣ㄱ-ㅎㅏ-ㅣ])(?P<kor>[가-힣ㄱ-ㅎㅏ-ㅣ])', '\g<notkor> \g<kor>', edit_text)
    # 연속적인 공백 제거
    edit_text = extra2(edit_text)
    
    return edit_text

In [546]:
# tokenizer split 1 : 연산자 <= < > >= 포함해서 띄어쓰기 구분해주기
def tok_split1(edit_text):
    
    # 띄어쓰기 양 옆에 삽입
    edit_text = re.sub('(?P<sym>[+-/*@])', ' \g<sym> ', edit_text)
    edit_text = re.sub('(?P<sym><|>|=)', ' \g<sym> ', edit_text)
    
    # 중복 적용된 >= <= 정정해주기
    edit_text = re.sub('<  =', '<=', edit_text)
    edit_text = re.sub('>  =', '>=', edit_text)
    edit_text = re.sub('-  >', '->', edit_text)
    
    # 연속적인 공백 제거
    edit_text = extra2(edit_text)
    
    return edit_text


# tokenizer split 2 : dictionary 토대로 좌우 split 해줌
def tok_split2(edit_text):
    for word in dict_sample1:
        # dict 에 있는 모든 단어 앞 뒤로 띄어쓰기
        replace = ' ' + word + ' '
        edit_text = re.sub(word, replace, edit_text)
        
    # tri angle --> triangle
    edit_text = re.sub('tri angle', 'triangle', edit_text)
    
    # oo 때문에 root --> r oo t
    edit_text = re.sub('r oo t', 'root', edit_text)
    
    # log 때문에 log_ --> log _
    edit_text = re.sub('log _', 'log_ ', edit_text)
    
    # 연속적인 공백 제거
    edit_text = extra2(edit_text)
    
    return edit_text

# tokenizer split 3 : 숫자 # 에 대해서 띄어쓰기
def tok_split3(edit_text):
    
    edit_text = re.sub('#', ' # ', edit_text)
    
    return edit_text

# tokenizer split 4 : 차수에 대한 공백 정리
def tok_split4(edit_text):
    
    # ^ 과 숫자는 띄어져있지 않도록
    edit_text = re.sub('\^\s', '^', edit_text)
    
    # ^- 뒤에 숫자든 뭐든 붙어 있도록~
    edit_text = re.sub('\^\- ', '^-', edit_text)
    
    # 하지만 앞과 ^은 띄어져 있도록~
    edit_text = re.sub('\^', ' ^', edit_text)
    
    
    return edit_text

# tokenizer split 5 : {::} _ 는 {::}_ 로 붙이기
def tok_split5(edit_text):
    
    edit_text = re.sub('\{\:\:\} \_', '{::}_', edit_text)
    
    return edit_text

# tokenizer split 6 : 숫자에 대해서도 띄어쓰기 보장
# 숫자는 무조건 개별로? 아니면 연속적인 숫자들은 한 묶음으로? : 일단 묶음으로
def tok_split6(edit_text):
    
    edit_text = re.sub('(?P<num>[0-9]+)', ' \g<num> ', edit_text)
    edit_text = extra2(edit_text)
    
    return edit_text

# tokenizer split 2 : dictionary 토대로 좌우 split 해줌
# 2.1 : dict_sample3 에 대해서도 추가
def tok_split2_1(edit_text):
    for word in dict_sample1:
        # dict 에 있는 모든 단어 앞 뒤로 띄어쓰기
        replace = ' ' + word + ' '
        edit_text = re.sub(word, replace, edit_text)
        
    for i in range(len(dict_sample3)):
        replace = ' ' + dict_sample3[i][0] + ' '
        edit_text = re.sub(dict_sample3[i][0], replace, edit_text)
        
    # tri angle --> triangle
    edit_text = re.sub('tri angle', 'triangle', edit_text)
    
    # oo 때문에 root --> r oo t
    edit_text = re.sub('r oo t', 'root', edit_text)
    
    # log 때문에 log_ --> log _
    edit_text = re.sub('log _', 'log_ ', edit_text)
    
    # 연속적인 공백 제거
    edit_text = extra2(edit_text)
    
    return edit_text

In [291]:
# noise delete 1 : $ 제거
def noise_del1(edit_text):
    
    edit_text = re.sub('\$',' ', edit_text)
    
    return edit_text


# noise delete 2 : # 제거
def noise_del2(edit_text):
    
    edit_text = re.sub('\\\\', ' ', edit_text)
    
    return edit_text

# noise delete 3 : # tarc 제거
def noise_del3(edit_text):
    
    edit_text = re.sub('tarc', ' ', edit_text)
    
    return edit_text

# noise delete 4 : # 잡 char 제거
def noise_del4(edit_text):
    
    for word in dict_delete:
        
        edit_text = re.sub(word, ' ', edit_text)
    
    return edit_text

# noise delete 5 : # begin{cases} end{cases} 제거 할까 말까 ㅎㅎ
def noise_del5(edit_text):
    
    for word in dict_case:
        
        edit_text = re.sub(word, ' ', edit_text)
    
    return edit_text

# noise delete 6 : {::} 제거  -  이거는 open data 에 해당
def noise_del6(edit_text):
    
    edit_text = re.sub('\{\:\:\}', ' ', edit_text)
    
    return edit_text


# noise delete 7 : _ 제거 - 둘 다 해당
def noise_del7(edit_text):
    
    edit_text = re.sub('_', ' ', edit_text)
    
    return edit_text

In [285]:
# noise substitute 1 : dfrac 에 대해서 나눗셈으로 변환
def noise_sub1(edit_text):
    
    edit_text = re.sub('dfrac(?P<word>.+?)\}\s+?\{','\g<word>} / {', edit_text)
    
    return edit_text


# noise substitute 2 : dictionary에서 서로 일치하는 형식으로 변환
def noise_sub2(edit_text):
    
    for (hidden_word, open_word) in dict_sample2:
        
        edit_text = re.sub(hidden_word, open_word, edit_text)
    
    return edit_text


# noise substitute 3 : dictionary에서 서로 일치하는 형식으로 변환 (역변환)
# open data 에다가 적용해야함! 
# hidden data 2 에다가도 다향한 형식으로 쓰인 게 있을 수 있으니 적용
def noise_sub3(edit_text):
    
    for (hidden_word, open_word) in dict_sample3:
        
        edit_text = re.sub(open_word, hidden_word, edit_text)
    
    return edit_text


# noise substitute 4 : 간혹 다른 형식으로 있는 숫자들에 대해서 통일
# 개별 숫자가 중요한지는 나중에 생각하자
def noise_sub4(edit_text):
    
    for (hidden_word, open_word) in dict_number:
        
        edit_text = re.sub(hidden_word, open_word, edit_text)
    
    return edit_text

In [576]:
# 완전 대충 순서:

# 괄호 통일하고
# 괄호 없애고
# 수학 기호 통일
# 등등

# 구문자 제거
# 공백 정리
# 차수에 대한 공백 double 정리
# 숫자 변환 에서 차이점을 둠
# ex1 / ex2 / ex3 / ex4 / ex5
# 변환 차수 보존 / 숫자 제거 차수 보존 / 숫자 제거 / 한글 제거 (ex3에서 이어서) / 수학 제거
#
# 차후 괄호에 대해서.....132$1/2$!3
# 차후 ne4에서 숫자 안 없어지는 거 시도해보자

def noisy_example0(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    
    #edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4((math_sym6(math_sym1(math_sym2(edit_text))))))))
    #edit_text = tok_kor2(math_sym3_1(kor_punc2(edit_text)))
    
    # 위 두개랑 이거랑 바꾸면 괄호에 대해서 보존 
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor2(math_sym3_1(kor_punc4(edit_text)))
    
    return edit_text

def noisy_example1(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor2(math_sym3_1(math_num5_1(kor_punc4(edit_text))))

    return edit_text


def noisy_example2(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor2(math_sym3_1(math_num6_1(kor_punc4(edit_text))))

    return edit_text


def noisy_example3(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor2(math_sym3_1(math_num8(kor_punc4(edit_text))))

    return edit_text


def noisy_example4(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = extra2(kor_lang1(math_sym3_1(math_num8(kor_punc4(edit_text)))))

    return edit_text


def noisy_example5(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(math_sym4(math_sym4_1(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(edit_text)
    edit_text = tok_kor2(kor_punc4(edit_text))

    return edit_text


# 형태소 분석기 적용!
# + 모든 수학식을 토크나이져 하려는 노력..!

def noisy_example0_morphs(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    
    #edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4((math_sym6(math_sym1(math_sym2(edit_text))))))))
    #edit_text = tok_kor2(math_sym3_1(kor_punc2(edit_text)))
    
    # 위 두개랑 이거랑 바꾸면 괄호에 대해서 보존 
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor1(math_sym3_1(kor_punc4(edit_text)))
    edit_text = example5_2(edit_text)
    
    return edit_text

def noisy_example1_morphs(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor1(math_sym3_1(math_num5_1(kor_punc4(edit_text))))
    edit_text = example5_2(edit_text)

    return edit_text


def noisy_example2_morphs(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor1(math_sym3_1(math_num6_1(kor_punc4(edit_text))))
    edit_text = example5_2(edit_text)

    return edit_text

def noisy_example3_morphs(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = tok_kor1(math_sym3_1(math_num8(kor_punc4(edit_text))))
    edit_text = example5_2(edit_text)

    return edit_text

def noisy_example4_morphs(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(tok_split4(tok_split1(tok_split4(math_sym7(math_sym6(math_sym1(math_sym2(edit_text))))))))
    edit_text = extra2(kor_lang1(math_sym3_1(math_num8(kor_punc4(edit_text)))))
    edit_text = example5_2(edit_text)

    return edit_text

def noisy_example5_morphs(edit_text):
    edit_text = extra3(edit_text)
    edit_text = noise_del7(noise_del5(noise_del4(math_sym4_1(math_sym4(noise_sub4(noise_sub3(noise_sub2(noise_sub1(edit_text)))))))))
    edit_text = noise_del3(edit_text)
    edit_text = math_sym5(noise_del1(noise_del2(edit_text)))
    edit_text = tok_split2_1(edit_text)
    edit_text = tok_kor1(kor_punc4(edit_text))
    edit_text = example5_2(edit_text)
    
    return edit_text

In [574]:
#1. 깔끔하게 다듬지만 구두점 숫자 변환 X
#2. 온점 반점 제거, 숫자는 차수에 대해서만 보존 나머지는 '#' 변환
#2.1 차수 아닌 숫자는 제거
#2.2 모든 숫자 제거
#3. 모든 구두점 제거, 숫자 제거
#4. only 한글
#5. tokenizer 적용

#6. 수학만 남음 : 한글 다 제거, 수학은 보기 좋게 구분 가능하도록

def example1(edit_text):
    edit_text = extra1(extra2(extra3(edit_text)))
    edit_text = kor_punc1(edit_text)
    edit_text = extra2(math_num1(math_sym5(math_sym2(math_sym3(edit_text))))) 
    
    return edit_text

def example2(edit_text):
    edit_text = extra1(extra2(extra3(edit_text)))
    edit_text = kor_punc2(edit_text)
    edit_text = extra2(math_num5_1(math_sym5(math_sym2(math_sym3(edit_text))))) 
    
    return edit_text

def example2_1(edit_text):
    edit_text = extra1(extra2(extra3(edit_text)))
    edit_text = kor_punc2(edit_text)
    edit_text = extra2(math_num6_1(math_sym5(math_sym2(math_sym3(edit_text)))))

    return edit_text

def example2_2(edit_text):
    edit_text = extra1(extra2(extra3(edit_text)))
    edit_text = kor_punc2(edit_text)
    edit_text = extra2(math_num8(math_sym5(math_sym2(math_sym3(edit_text)))))

    return edit_text

def example3(edit_text):
    edit_text = extra1(extra2(extra3(edit_text)))
    edit_text = kor_punc3(edit_text)
    edit_text = extra2(math_num8(math_sym5(math_sym2(math_sym3(edit_text))))) 
    
    return edit_text

def example4(edit_text):
    edit_text = extra1(extra2(extra3(edit_text)))
    edit_text = kor_punc4(edit_text)
    edit_text = extra2(math_sym5(math_sym4(edit_text)))
    
    return edit_text

def example5(edit_text):
    
    # 1 : 쓸때없는 것들 제거
    edit_text = extra3(edit_text)
    
    # 2 : div divide xx times 등 기본 연산자 형식 통일
    # 모든 괄호 유형 통일 
    # 수학 공식을 두르는 ` 제거
    edit_text = math_sym5(math_sym6(math_sym1(math_sym2(edit_text))))
    
    # 3 : 차수 보존 나머지 숫자 #으로 변환
    edit_text = math_num5_1(edit_text)
    
    # 4 : 한글 부분에서 온점, 반점만 제거
    edit_text = kor_punc2(edit_text)
    
    # 5 : 괄호 안에 공백 보장 
    edit_text = extra4(edit_text)
    
    # 6 : 연산자와 dictionary 그리고 # 대해서 띄어쓰기 
    # 차수에 대한 공백 정리
    # {::} _ 는 {::}_ 로 붙이기
    # 연속적인 공백 제거
    edit_text = tok_split4(tok_split3(tok_split2(tok_split1(edit_text))))
    
    # 7 : 한글 부분에 대해서만 Komoran() 적용
    edit_text = tok_kor1(edit_text)
    
    return edit_text


def example6(edit_text):
    edit_text = extra1(extra2(extra3(edit_text)))
    edit_text = kor_lang1(kor_punc4(edit_text)) # 괄호까지 제거
    edit_text = extra2(math_num1(math_sym5(math_sym2(math_sym1(edit_text)))))
    edit_text = tok_split2(tok_split1(edit_text))
    
    return edit_text

def example7(edit_text):
    
    edit_text = extra2(noise_del1(noise_del2(noise_sub2(noise_sub1(test2)))))
    
    return edit_text

def example8(edit_text):
    
    edit_text = extra2(noise_del2(noise_sub2(noise_sub1(test2))))

    return edit_text

def example9(edit_text):
    
    edit_text = extra2(noise_del1(noise_sub2(noise_sub1(test2))))
    
    return edit_text

def example10(edit_text):
    
    edit_text = extra2(noise_sub2(noise_sub1(test2)))
    
    return edit_text

def example_2match_noisy(edit_text):
    # might not be needed :)RT@!%#
    return edit_text

def example5_1(edit_text):

    # 5 : 괄호 안에 공백 보장 
    edit_text = extra4(edit_text)
    
    # 6 : 연산자와 dictionary 그리고 # 대해서 띄어쓰기 
    # 차수에 대한 공백 정리
    # {::} _ 는 {::}_ 로 붙이기
    # 연속적인 공백 제거
    edit_text = tok_split4(tok_split3(tok_split2(tok_split1(edit_text))))
    
    # 7 : 한글 부분에 대해서만 Komoran() 적용
    edit_text = tok_kor1(edit_text)
    
    # 8 : 숫자에 대해서도 띄어쓰기 해주기
    edit_text = tok_split6(edit_text)
    
    return edit_text

def example5_2(edit_text):

    # 5 : 괄호 안에 공백 보장 
    edit_text = extra4(edit_text)
    
    # 6 : 연산자와 dictionary 그리고 # 대해서 띄어쓰기 
    # 차수에 대한 공백 정리
    # {::} _ 는 {::}_ 로 붙이기
    # 연속적인 공백 제거
    edit_text = tok_split4(tok_split3(tok_split2(tok_split1(edit_text))))
    
    # 7 : 한글 부분에 대해서만 Komoran() 적용
    edit_text = tok_kor1(edit_text)
    
    # 8 : 숫자에 대해서도 띄어쓰기 해주기
    edit_text = tok_split6(edit_text)
    
    edit_text = tok_split2_1(edit_text)
    
    edit_text = extra2(edit_text)
    
    return edit_text